In [13]:
import cv2
import numpy as np
from pupil_apriltags import Detector
import serial
import time


# Define the serial port and baud rate
ser = serial.Serial('COM4', 9600)  # Replace 'COMX' with your Arduino's COM port

# Delay to allow time for the serial connection to be established
time.sleep(2)

# Load camera calibration parameters
data = np.load('camera_calibration.npz')
camera_matrix = data['camera_matrix']
distortion_coefficient = data['distortion_coefficient']

# Initialize AprilTag detector
at_detector = Detector(families='tag36h11',
                       nthreads=4,
                       quad_decimate=1.0,
                       quad_sigma=0.0,
                       refine_edges=1,
                       decode_sharpening=0.25,
                       debug=0)

# Start webcam capture
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect AprilTags in the image
    tags = at_detector.detect(gray, estimate_tag_pose=True, camera_params=[camera_matrix[0,0], camera_matrix[1,1], camera_matrix[0,2], camera_matrix[1,2]], tag_size=0.165) # tag_size should be set to the real size of the tag in meters

    for tag in tags:
        # Draw a bounding box around the detected tag
        cv2.polylines(frame, [np.array(tag.corners, dtype=np.int32).reshape((-1,1,2))], True, (0,255,0), 2)

        # Display the tag ID on the image
        cv2.putText(frame, str(tag.tag_id), (int(tag.center[0]), int(tag.center[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Calculate distance from the camera (assuming tvec is in meters)
        distance = np.linalg.norm(tag.pose_t)
        print(f"Distance to tag {tag.tag_id}: {distance} meters")

        if distance > 3:
            # Set the servo angle to 60 degrees
            speed = 30  
            slow = 18
            speed_string = f"{speed},{slow}\n"
            ser.write(speed_string.encode())
            time.sleep(1)
        else:
            # Set the servo angle to 0 degrees
            speed = 0  # Example speed value for motor 1
            slow = 0  # Example speed value for motor 2
            speed_string = f"{speed},{slow}\n"
            ser.write(speed_string.encode())
            time.sleep(1)
        
          

    # Display the resulting frame
    cv2.imshow('Frame', frame)

    # Break the loop with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


Distance to tag 0: 1.9993519491995906 meters
Distance to tag 0: 2.1143242817362404 meters
Distance to tag 0: 2.539944232913946 meters
Distance to tag 0: 2.9744171543914164 meters
Distance to tag 0: 3.418735069295784 meters
Distance to tag 0: 3.7752297298928412 meters
Distance to tag 0: 3.966885514206784 meters
Distance to tag 0: 3.948346406730888 meters
Distance to tag 0: 3.8592629611930036 meters
Distance to tag 0: 3.1539844356186477 meters
Distance to tag 0: 2.49270731906891 meters
Distance to tag 0: 1.9114410944477227 meters
Distance to tag 0: 1.757037615763064 meters
Distance to tag 0: 2.091476429732199 meters
Distance to tag 0: 2.752994432243172 meters
Distance to tag 0: 3.1971621719132206 meters
Distance to tag 0: 3.219905628818786 meters
Distance to tag 0: 3.249305735525336 meters
Distance to tag 0: 2.799820606728197 meters
Distance to tag 0: 2.2740741306569396 meters
Distance to tag 0: 2.237559666134482 meters
Distance to tag 0: 2.7077206109441 meters
Distance to tag 0: 3.09789